In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory

from pyomo_models.base import BaseModel
from util.scenario import Scenario

# Base model

In [ ]:
sc = Scenario("scenarios/single_drone.yaml")

_, ax = plt.subplots(figsize=(7,7), dpi=80)
sc.plot(ax=ax)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
# PARAMETERS
B_min = 0
B_max = 1
B_start = [1]
v = [1]
r_charge = np.array([0.1])
r_deplete = np.array([0.25])

parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
model = BaseModel(sc, parameters)

In [ ]:
# solution = SolverFactory('cbc').solve(model, tee=True) 
solution = SolverFactory('cbc').solve(model) 
time_taken = solution['solver'][0]['time']
print(f"Took {time_taken:.3f}s to find charging schedule")

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.C[:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")

In [ ]:
axes = plt.subplots(1,2, figsize=(15,30), dpi=80) 
ax1, ax2= axes[1]

# problem
sc.plot(ax=ax1)
ax1.set_aspect('equal', adjustable='box')
ax1.axis('off')

model.plot(ax=ax2)
# plot_second_solved(model, positions_S, positions_w, ax=ax2)
ax2.set_aspect('equal', adjustable='box')
ax2.axis('off')

plt.show()

# Model definition (two drones)

In [ ]:
# PARAMETERS
B_min = 0
B_max = 1
B_start = [1, 1]
v = [1, 1]
r_charge = np.array([0.1, 0.1])
r_deplete = np.array([0.3, 0.3])

sc = Scenario("scenarios/two_drones.yml")

In [ ]:
_, ax = plt.subplots(figsize=(7,7), dpi=80)
sc.plot(ax=ax)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
model = BaseModel(sc, parameters)

In [ ]:
# solution = SolverFactory('cbc').solve(model, tee=True) 
solution = SolverFactory('gurobi').solve(model) 
time_taken = solution['solver'][0]['time']
print(f"Took {time_taken:.3f}s to find charging schedule")

In [ ]:
axes = plt.subplots(1,2, figsize=(15,30), dpi=80) 
ax1, ax2= axes[1]

# problem
sc.plot(ax=ax1)
ax1.set_aspect('equal', adjustable='box')
ax1.axis('off')

model.plot(ax=ax2)
ax2.set_aspect('equal', adjustable='box')
ax2.axis('off')

plt.show()

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.C[:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")